# get numeration int addition response from gemini

Install necessary packages

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [9]:
with open('../../data/numeration_int_addn.json','r') as file:
    data = json.load(file)

# create batches, batch sizes is heuristics 
# batch size is limited by output token limit which is 2048

size_dict = {'d1':10,
             'd2':10, # big sizes are not working, model gives blank outputs
             'd3':20,
             'd4':20,
             'd5':20,
             'd6':10,
             'd7':10,}


batch_prompts = []

for sign_of_both in data:
    # sign of both : both positive, word neg digit pos, both negative
    for type_of_both in data[sign_of_both]:
        # type of both : dw, ww
        for digit in data[sign_of_both][type_of_both]:
            # digit : d1, d2, d3, d4, d5, d6, d7
            list_data = data[sign_of_both][type_of_both][digit]
            # get batch size
            BATCH_SIZE = size_dict[digit]
            
            index = 0

            while index< len(list_data):
                batch_prompt = list_data[index:min(index+BATCH_SIZE,len(list_data))]
                batch_prompts.append(batch_prompt)
                index += BATCH_SIZE
    
print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])

Number of batches: 470
Number of prompts in each batch: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

### conduct experiment for output sum in digit form

eg. <br>
2 + three   = 5<br>
two + 3     = 5<br>
two + three = 5<br>

In [13]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you some addition questions. The two numbers I ask you to add can be in either digit form or word form, irrespective of the input form I want you to answer in digit form. For example, if I ask 'What is 2 + three?', you should answer '2 + three = 5'. Note that input format should be preserved in answer. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [14]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you some addition questions. The two numbers I ask you to add can be in either digit form or word form, irrespective of the input form I want you to answer in digit form. For example, if I ask 'What is 2 + three?', you should answer '2 + three = 5'. Note that input format should be preserved in answer. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I will add the two numbers you give me, and return the answer in digit form, preserving the input format. For example, if you ask \'What is 2 + three?\', I will answer \'2 + three = 5\'."
}
role: "model"
]


In [15]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation
with open('initial_messages_numeration_int_add_op_digit.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

In [16]:
# path to save responses
response_file = 'gemini_response/Numeration_int_addition_digit_op_response.txt'

# request per minute limit for free api
rpm = 60

In [18]:
# to start from a specific batch number, give printed_batch_num -1 here
batch_no_to_start_from = 0

for i, batch_prompt in enumerate(batch_prompts):

    # to start from a specific batch in case of errors
    if i < batch_no_to_start_from:
        continue
    
    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"what is {a} + {b} ? \n " for (a,b,c,d) in batch_prompt])
    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        print("Error in response")
        model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1 of 470
# of prompts:  10 # of responses:  10
[0, 'zero', 'zero', 0] ->0 + zero = 0
[0, 'one', 'one', 1] ->0 + one = 1
[0, 'two', 'two', 2] ->0 + two = 2
['zero', 3, 'three', 3] ->zero + 3 = 3
['zero', 4, 'four', 4] ->zero + 4 = 4
['zero', 5, 'five', 5] ->zero + 5 = 5
['zero', 6, 'six', 6] ->zero + 6 = 6
[0, 'seven', 'seven', 7] ->0 + seven = 7
['zero', 8, 'eight', 8] ->zero + 8 = 8
[0, 'nine', 'nine', 9] ->0 + nine = 9
Received responses for batch 1 of 470
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Sending batch 2 of 470
# of prompts:  10 # of responses:  10
['one', 0, 'one', 1] ->one + 0 = 1
[1, 'one', 'two', 2] ->1 + one = 2
['one', 2, 'three', 3] ->one + 2 = 3
[1, 'three', 'four', 4] ->1 + three = 4
[1, 'four', 'five', 5] ->1 + four = 5
[1, 'five', 'six', 6] ->1 + five = 6
[1, 'six', 'seven', 7] ->1 + six = 7
[1, 'seven', 'eight', 8] ->1 + seven = 8
[1, 'eight', 'nine', 9] ->1 + eight = 9
[1, 'nine', 'ten', 10] ->1 + nine = 10
Received resp

### conduct experiment for output sum in word form

eg. <br>
2 + three   = five<br>
two + 3     = five<br>
two + three = five<br>

In [19]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you some addition questions. The two numbers I ask you to add can be in either digit form or word form, irrespective of the input form I want you to answer in word form. For example, if I ask 'What is 2 + three?', you should answer '2 + three = five'. Note that input format should be preserved in answer. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [20]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you some addition questions. The two numbers I ask you to add can be in either digit form or word form, irrespective of the input form I want you to answer in word form. For example, if I ask 'What is 2 + three?', you should answer '2 + three = five'. Note that input format should be preserved in answer. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand your instructions. I will wait for your addition questions and I will answer each of them in word form, preserving the input format.\n\nFor example, if you ask \'What is 2 + three?\', I will answer \'2 + three = five\'.\n\nLet\'s begin!"
}
role: "model"
]


In [21]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation
with open('initial_messages_numeration_int_add_op_word.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

In [22]:
# path to save responses
response_file = 'gemini_response/Numeration_int_addition_word_op_response.txt'

# request per minute limit for free api
rpm = 60

In [23]:
# to start from a specific batch number, give printed_batch_num -1 here
batch_no_to_start_from = 0

for i, batch_prompt in enumerate(batch_prompts):

    # to start from a specific batch in case of errors
    if i < batch_no_to_start_from:
        continue
    
    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"what is {a} + {b} ? \n " for (a,b,c,d) in batch_prompt])
    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        print("Error in response")
        model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1 of 470
# of prompts:  10 # of responses:  10
[0, 'zero', 'zero', 0] ->0 + zero = zero
[0, 'one', 'one', 1] ->0 + one = one
[0, 'two', 'two', 2] ->0 + two = two
['zero', 3, 'three', 3] ->zero + 3 = three
['zero', 4, 'four', 4] ->zero + 4 = four
['zero', 5, 'five', 5] ->zero + 5 = five
['zero', 6, 'six', 6] ->zero + 6 = six
[0, 'seven', 'seven', 7] ->0 + seven = seven
['zero', 8, 'eight', 8] ->zero + 8 = eight
[0, 'nine', 'nine', 9] ->0 + nine = nine
Received responses for batch 1 of 470
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Sending batch 2 of 470
# of prompts:  10 # of responses:  10
['one', 0, 'one', 1] ->one + 0 = one
[1, 'one', 'two', 2] ->1 + one = two
['one', 2, 'three', 3] ->one + 2 = three
[1, 'three', 'four', 4] ->1 + three = four
[1, 'four', 'five', 5] ->1 + four = five
[1, 'five', 'six', 6] ->1 + five = six
[1, 'six', 'seven', 7] ->1 + six = seven
[1, 'seven', 'eight', 8] ->1 + seven = eight
[1, 'eight', 'nine', 9] ->1 + eight = 